【待精進】
   
*   如何下搜尋關鍵字


#**套件部署**

In [ ]:
import re
import time
import random
import requests
from datetime import datetime

from lxml import etree
from lxml.html import fromstring

import pandas as pd
import numpy as np

In [ ]:
%pip install ckiptagger

# -*- coding: utf-8 -*-
from ckiptagger import WS, POS, NER

from ckiptagger import data_utils
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
1.88GB [00:14, 133MB/s]


# **篩選資料集**

In [ ]:
laws = pd.DataFrame({
    '妨害風化罪': ['罰金 網路 妨害風化罪', '拘役 網路 妨害風化罪'],
    '恐嚇危害安全罪':['罰金 網路 恐嚇危害安全罪', '拘役 網路 恐嚇危害安全罪'],
    '公然侮辱罪': ['罰金 網路 公然侮辱罪', '拘役 網路 公然侮辱罪'],
    '誹謗罪':['罰金 網路 誹謗罪', '拘役 網路 誹謗罪'],
    '妨害秘密罪':['罰金 網路 妨害秘密罪', '拘役 網路 妨害秘密罪'],
    '妨害電腦使用罪':['罰金 網路 妨害電腦使用罪', '拘役 網路 妨害電腦使用罪']
})

#laws.shape[1]

#**函式**

###**爬蟲**

In [ ]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
}

In [ ]:
def get_news_list(keyword, page_num):
    base_url = "https://udn.com/api/more"

    news_list = []
    for page in range(page_num):
        channelId = 2
        type_ = 'searchword'
        query = f"page={page+1}&id=search:{keyword}&channelId={channelId}&type={type_}&last_page={page+1}"
        news_list_url = base_url + '?' + query
        # print(news_list_url)
        # https://udn.com/api/more?page=2&id=search:網路%20公然侮辱&channelId=2&type=searchword&last_page=6
        
        r = requests.get(news_list_url, headers=HEADERS)
        news_data = r.json()

        for items in news_data['lists']:  #判斷社會新聞
          if 'news/story/7321' in items['titleLink']:
            news_list.append(items)
          else:
            pass

        time.sleep(random.uniform(1, 2))

    return news_list

#get_news_list('罰金 網路 公然侮辱罪', 1)

In [ ]:
#def get_news_content(news_list):
#  content = []
#  for items in news_list:
#    if 'https://udn.com/news/story/' in items['titleLink']:
#      cont_r = requests.get(items['titleLink'], headers=HEADERS)
#      cont_data = etree.HTML(cont_r.content.decode('utf-8'))
#      cont = [etree.tostring(tar, method='text', encoding='unicode') for tar in cont_data.xpath("/html/body/main/div/section[2]/section/article/div/section[1]/p")]
       
#      cont_str = ""
#      for i in range(len(cont)):
#        try:
#          cont_str = cont_str + str(cont[i*2])
#        except:
#          pass
 
#      content.append(cont_str)

#    else:
#      content.append(items['titleLink'])

#  return content

In [ ]:
def news_crawler(law, keyword, page_num):
  news_list = get_news_list(keyword, page_num)
#  content = get_news_content(news_list)

  df_list = pd.DataFrame(news_list)
  times = []
  for items in df_list['time']:
    times.append(str(items).split("'")[3])
  
  df = pd.DataFrame({
      "law": law,
      "title": df_list['title'],
      "link": df_list['titleLink'],
      "cont": df_list['paragraph'],
      "time": times
  })

  return df

#news_crawler('公然侮辱罪', '拘役 網路 公然侮辱罪', 1)

### **資料獲取**

In [ ]:
def execute(law_option, page_num):
  exe_raw = pd.DataFrame()

  for keyword in laws[law_option]:
    try:
      exe_raw = pd.concat([exe_raw, news_crawler(law_option, keyword, page_num)], axis=0)
    except:
      pass

  try:
    exe = exe_raw.drop_duplicates(subset=['title'])
    exe.index = range(len(exe))
  except:
    pass

  return exe

### **刪除重複案件**

In [ ]:
def remove_prefix(str_list):
  reSortedList = sorted(str_list, key=len)
  index = 1
  result = []

  for char in reSortedList:
    result.append(char)
    for otherChar in reSortedList[index:]:
      match = otherChar.find(char)
      if match == 0:
        result.pop()
        break

    index += 1

  return result

#remove_prefix(['王', '王北北', '嘻嘻', '北王'])

In [ ]:
def ckip(cont):
  ws = WS("./data")
  pos = POS("./data")
  ner = NER("./data")

  ws_results = []
  pos_results = []

  for cont in cont:
    ws_cont = ws([cont])
    pos_cont = pos(ws_cont)
    ws_results.append(ws_cont)
    pos_results.append(pos_cont)

  df_c = pd.DataFrame({
      'ws': ws_results,
      'pos': pos_results
  })


  nb_result = []
  na_result = []
  for i in range(len(df_c)):
    new_word = [word for sentence in df_c.ws[i] for word in sentence]
    new_pos = [pos for sentence in df_c.pos[i] for pos in sentence]

    df_c1 = pd.DataFrame({"word": new_word, 'pos': new_pos})
    
    # nb
    nb1 = []
    for w in df_c1['word'][df_c1['pos']=='Nb']:
      w = re.sub('[\u59d3|\u5973|\u7537]$', '', w)
      nb1.append(w)

    nb2 = remove_prefix(nb1)
    nb_result.append(nb2)

    # na
    na1 = []
    for w in df_c1['word'][df_c1['pos']=='Na']:
      w = re.sub('[\u59d3|\u5973|\u7537]$', '', w)
      na1.append(w)

    na2 = remove_prefix(na1)
    na_result.append(na2)


  df_c_result = pd.DataFrame({
      'nb': nb_result,
      'na': na_result,
      #'keyname': nb_result + na_result
  })
  return df_c_result


#ckip(exe['cont'])

In [ ]:
def exe_de_dup(law_option, page_num):
  exe = execute(law_option, page_num)
  ckip_result = ckip(exe['cont'])
  de_dup = exe.merge(ckip_result, how='right', left_index=True, right_index=True)
  de_dup.sort_values('time', inplace=True)
  de_dup = de_dup.reset_index(drop=True)

  de_result = pd.DataFrame()
  index = 1

  for row in range(len(de_dup)):
    de_result = pd.concat([de_result, de_dup[row:row+1]], axis=0)

    for target in de_dup['nb'][index:]:
      match_nb = [i for i in de_dup['nb'][row] if i in target]
      if len(match_nb) > 1:
        #print('nb', de_result[-1:], match_nb)
        de_result = de_result[:-1]
        break
      elif len(match_nb) == 1:
        #print('na', de_dup['nb'][row], target)
        match_na1 = [j for j in de_dup['na'][row] if j in target]
        match_na2 = [j for j in de_dup['nb'][row] if j in de_dup['na'][index]]
        if (len(match_na1) > 0 and match_na1 not in match_nb) or (len(match_na2) > 0 and match_na2 not in match_nb):
          #print('na', de_result[-1:], match_na1, match_na2)
          de_result = de_result[:-1]
          break


    index += 1

  de_result.drop(['nb', 'na'], axis=1)
  de_result = de_result.reset_index(drop=True)
  return de_result


#exe_de_dup('公然侮辱罪', 1)

### **Google Sheet 輸出**

In [ ]:
# Google Sheet 操作語法參考：https://medium.com/@fsflyingsoar/%E7%AD%86%E8%A8%98-for-python-google-colab-gspread-a397e80d071d

def get_sheet(book_title, sheet_title):
  try:
    sh = gc.open(book_title)
  except:
    sh = gc.create(book_title)
  
  try: 
    worksheet = sh.worksheet(sheet_title)
    worksheet.clear()
  except:
    worksheet = sh.add_worksheet(title=sheet_title, rows='100', cols='5')

  return worksheet

In [ ]:
def save_sheet(exe, worksheet, count):
  worksheet.update_cell(count+2, 1, exe['law'][count])
  worksheet.update_cell(count+2, 2, exe['title'][count])
  worksheet.update_cell(count+2, 3, exe['link'][count])
  worksheet.update_cell(count+2, 4, exe['cont'][count])
  worksheet.update_cell(count+2, 5, exe['time'][count])

In [ ]:
def sheet_set(sheet_option, exe):
  if exe.empty:
      print('Invalid Syntax')

  else:
    worksheet = get_sheet('news', sheet_option)
    worksheet.update_cell(1, 1, 'law')
    worksheet.update_cell(1, 2, 'title')
    worksheet.update_cell(1, 3, 'link')
    worksheet.update_cell(1, 4, 'cont')
    worksheet.update_cell(1, 5, 'time')
    # print(worksheet)

    for count in range(len(exe)):
      try:
        save_sheet(exe, worksheet, count)
      except:
        pass

In [ ]:
def exe_to_sheet(page_num):
  all_exe = pd.DataFrame()

  for law in laws:
    exe = exe_de_dup(law, page_num)
    sheet_set(law, exe)
    all_exe = pd.concat([all_exe, exe], axis=0, ignore_index=True)

  sheet_set('all', all_exe)

#**執行**

In [ ]:
###### Google ######
# !pip install --upgrade gspread


###### Ckip ######
# %pip install tensorflow
# %pip install gdown

In [ ]:
# Google 身份驗證

import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
credentials = GoogleCredentials.get_application_default()

gc = gspread.authorize(credentials)

In [ ]:
# 新聞來源：UDN聯合新聞網

exe_to_sheet(1)

### 參數格式：法條, 新聞擷取頁數【新聞總筆數 = 參數*20筆*法條數】, 欲儲存資料表
### 請注意執行程式碼的頻率

/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
